# Pop vs Non-Pop Genre Classification
**by Kyle Furey & Dhruv Solanki**


## 1. Setup & Data Loading


In [ ]:
# Imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfl
from tensorflow.keras import callbacks as tfkc



In [ ]:
# Load behavioral Spotify dataset
DATA_PATH = '../data/spotify_final_with_behavior.csv'

print('Loading data from', DATA_PATH)
df = pd.read_csv(DATA_PATH)
print('Shape:', df.shape)
print('Columns:', df.columns.tolist())

# Define pop-vs-non-pop label using tags/genre (real Spotify metadata)
import re

pop_keyword_patterns = [
    r"\bpop\b",
    r"dance[- ]?pop",
    r"electro[- ]?pop",
    r"synth[- ]?pop",
    r"teen pop",
    r"pop rock",
    r"pop rap",
    r"latin pop",
    r"indie pop",
    r"k[- ]?pop",
    r"j[- ]?pop",
    r"c[- ]?pop",
]

def is_pop_track(row):
    # Genre column now contains the actual genres (was moved from tags column)
    text = f"{row.get('genre', '')}".lower()
    return int(any(re.search(pat, text) for pat in pop_keyword_patterns))

df['is_pop_genre'] = df.apply(is_pop_track, axis=1)
print('Pop class positive rate:', df['is_pop_genre'].mean())

# Define features and target based on pop labels
# Enhanced feature set with more numeric features
numeric_features = ['spotify_popularity', 'album_release_year', 'tempo_bpm_synth', 'position']
cat_feature = 'time_of_day_synth'
# REMOVED: genre TF-IDF (data leakage - contains 'pop' which directly indicates target)
# text_feature = 'genre'  # Genre column now contains the actual genres (was moved from tags)
text_feature = None  # Disabled to prevent data leakage
target_column = 'is_pop_genre'

# Extract additional features
df['is_explicit_binary'] = df['is_explicit'].astype(int)

# Extract temporal features from album_release_date
df['album_release_date'] = pd.to_datetime(df['album_release_date'], errors='coerce')
df['release_month'] = df['album_release_date'].dt.month.fillna(0).astype(int)
df['release_decade'] = (df['album_release_year'] // 10 * 10).astype(int)

# Create interaction features
df['popularity_x_year'] = df['spotify_popularity'] * df['album_release_year']
df['tempo_x_year'] = df['tempo_bpm_synth'] * df['album_release_year']

# Use derived features if they exist (created by create_derived_features.py)
# These are proxy features for audio features (danceability, energy, valence, acousticness)
# since Spotify deprecated the audio-features endpoint for new apps
derived_features = []
if 'is_highly_popular' in df.columns:
    derived_features.extend(['is_highly_popular', 'is_moderately_popular', 'popularity_normalized'])
# REMOVED: has_pop_genre and genre_count (data leakage - derived from same genre column as target)
# if 'has_pop_genre' in df.columns:
#     derived_features.extend(['has_pop_genre', 'genre_count'])
if 'is_recent' in df.columns:
    derived_features.extend(['is_recent', 'is_very_recent'])
if 'tempo_is_pop_range' in df.columns:
    derived_features.extend(['tempo_is_pop_range', 'tempo_normalized'])
if 'is_daytime' in df.columns:
    derived_features.append('is_daytime')
if 'is_not_explicit' in df.columns:
    derived_features.append('is_not_explicit')
if 'popular_recent' in df.columns:
    derived_features.append('popular_recent')
if 'mainstream_pop_signal' in df.columns:
    derived_features.append('mainstream_pop_signal')

print(f'Using {len(derived_features)} derived features as proxies for audio features')
if len(derived_features) > 0:
    print(f'Derived features: {derived_features[:5]}...' if len(derived_features) > 5 else f'Derived features: {derived_features}')

# Prepare feature sets - include derived features
X_num = df[numeric_features + ['is_explicit_binary', 'release_month', 'release_decade', 
                                'popularity_x_year', 'tempo_x_year'] + derived_features].copy()
X_cat = df[[cat_feature]].copy()
# X_text = df[text_feature].fillna('')
X_text = pd.Series([''] * len(df))  # Empty to prevent data leakage
y = df[target_column].astype(int).values

# One-hot encode time_of_day_synth
X_cat_dummies = pd.get_dummies(X_cat, columns=[cat_feature], drop_first=False)

# REMOVED: TF-IDF on genre (data leakage - contains 'pop' which directly indicates target)
# tfidf = TfidfVectorizer(max_features=15, ngram_range=(1, 1), min_df=20)
# X_tfidf = tfidf.fit_transform(X_text).astype('float32')
import scipy.sparse as sp
# Create empty TF-IDF matrix (no genre features to prevent leakage)
X_tfidf = sp.csr_matrix((len(X_text), 0), dtype='float32')

# Scale numeric features on the full dataset
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

# Build final feature matrix: numeric + one-hot time_of_day + TF-IDF tags
X_tab = pd.concat([
    pd.DataFrame(X_num_scaled, columns=X_num.columns),
    X_cat_dummies.reset_index(drop=True),
], axis=1)

import scipy.sparse as sp
X_full = sp.hstack([sp.csr_matrix(X_tab.values.astype('float32')), X_tfidf], format='csr')
feature_dim = X_full.shape[1]
print('Enhanced feature dimension (tabular + genre TF-IDF):', feature_dim)
print('Numeric features:', len(X_num.columns))
print('Categorical features (one-hot):', X_cat_dummies.shape[1])
print('TF-IDF features (from genre column):', X_tfidf.shape[1])

# Create explicit train / validation / test splits (stratified)
from sklearn.model_selection import train_test_split

X_train_full, X_temp, y_train_full, y_temp = train_test_split(
    X_full, y, test_size=0.3, random_state=42, stratify=y
)
X_val_full, X_test_full, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

X_train_np = X_train_full.toarray().astype('float32')
X_val_np = X_val_full.toarray().astype('float32')
X_test_np = X_test_full.toarray().astype('float32')
y_train = y_train_full

print('Train / Val / Test sizes:', X_train_np.shape[0], X_val_np.shape[0], X_test_np.shape[0])


## 2. Baseline Models


In [ ]:
# Build and train FFNN for pop genre prediction

input_dim = X_train_np.shape[1]
print('Input dimension:', input_dim)

model = tfk.Sequential([
    tfl.Input(shape=(input_dim,)),
    tfl.Dense(32, activation='relu'),
    tfl.Dropout(0.3),
    tfl.Dense(16, activation='relu'),
    tfl.Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer=tfk.optimizers.Adam(learning_rate=1e-3),
    loss=tfk.losses.BinaryCrossentropy(),
    metrics=[tfk.metrics.BinaryAccuracy(name='accuracy')]
)

model.summary()

# Early stopping on validation loss using held-out validation set
callbacks = [
    tfkc.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

history = model.fit(
    X_train_np, y_train,
    validation_data=(X_val_np, y_val),
    epochs=100,
    batch_size=256,
    verbose=1,
    callbacks=callbacks
)


In [ ]:
# Plot training & validation loss
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Val loss')
plt.xlabel('Epoch')
plt.ylabel('Binary cross-entropy')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()


In [ ]:
# Evaluate on test set
print('Evaluating on test set ...')

probs = model.predict(X_test_np).ravel()
preds = (probs >= 0.5).astype(int)

print('\nClassification report (0 = non-pop, 1 = pop):')
print(classification_report(y_test, preds, digits=3))

try:
    auc = roc_auc_score(y_test, probs)
    print(f'ROC AUC: {auc:.3f}')
except Exception as e:
    print('Could not compute ROC AUC:', e)

cm = confusion_matrix(y_test, preds)
ConfusionMatrixDisplay(cm, display_labels=['Non-pop', 'Pop']).plot()
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Keras training & validation accuracy curves

if 'accuracy' in history.history and 'val_accuracy' in history.history:
    plt.figure(figsize=(6,4))
    plt.plot(history.history['accuracy'], label='Train accuracy')
    plt.plot(history.history['val_accuracy'], label='Val accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Keras FFNN Training vs Validation Accuracy')
    plt.legend()
    plt.show()
else:
    print('Accuracy metrics not available in Keras history.')


## 3. Improved Models


### Improved Keras FFNN


In [ ]:
# Compute class weights for imbalanced labels
from sklearn.utils import class_weight

classes = np.unique(y_train)
weights = class_weight.compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = {int(c): float(w) for c, w in zip(classes, weights)}
print('Class weights:', class_weight_dict)

# Build improved FFNN with BatchNorm, Dropout, and L2 regularization
reg = tfk.regularizers.l2(1e-3)  # Stronger L2 regularization to prevent overfitting
input_dim = X_train_np.shape[1]

improved_model = tfk.Sequential([
    tfl.Input(shape=(input_dim,)),
    tfl.Dense(64, activation='relu', kernel_regularizer=reg),
    tfl.BatchNormalization(),
    tfl.Dropout(0.5),  # Increased dropout to prevent overfitting
    tfl.Dense(32, activation='relu', kernel_regularizer=reg),
    tfl.BatchNormalization(),
    tfl.Dropout(0.5),  # Increased dropout to prevent overfitting
    tfl.Dense(16, activation='relu', kernel_regularizer=reg),
    tfl.Dense(1, activation='sigmoid'),
])

improved_model.compile(
    optimizer=tfk.optimizers.Adam(learning_rate=1e-3),
    loss=tfk.losses.BinaryCrossentropy(),
    metrics=[tfk.metrics.BinaryAccuracy(name='accuracy')]
)

improved_model.summary()

callbacks_improved = [
    tfkc.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),  # More aggressive early stopping
    tfkc.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1),
]

history_improved = improved_model.fit(
    X_train_np, y_train,
    validation_data=(X_val_np, y_val),
    epochs=100,
    batch_size=256,
    verbose=1,
    callbacks=callbacks_improved,
    class_weight=class_weight_dict,
)

# Plot loss and accuracy for the improved model
plt.figure(figsize=(6,4))
plt.plot(history_improved.history['loss'], label='Train loss (improved)')
plt.plot(history_improved.history['val_loss'], label='Val loss (improved)')
plt.xlabel('Epoch')
plt.ylabel('Binary cross-entropy')
plt.title('Improved Keras FFNN Loss')
plt.legend()
plt.show()

if 'accuracy' in history_improved.history:
    plt.figure(figsize=(6,4))
    plt.plot(history_improved.history['accuracy'], label='Train acc (improved)')
    plt.plot(history_improved.history.get('val_accuracy', []), label='Val acc (improved)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Improved Keras FFNN Accuracy')
    plt.legend()
    plt.show()


In [ ]:
# Threshold tuning for improved model (optimize F1 for pop class)
from sklearn.metrics import precision_recall_fscore_support

probs_imp = improved_model.predict(X_test_np).ravel()

best_thresh = 0.5
best_f1 = -1.0
best_stats = None
thresholds = np.linspace(0.1, 0.9, 17)

for thr in thresholds:
    preds_thr = (probs_imp >= thr).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, preds_thr, average='binary', zero_division=0
    )
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = thr
        best_stats = (precision, recall, f1)

print(f'Best threshold (by F1 for pop class): {best_thresh:.2f}')
print(f'Precision: {best_stats[0]:.3f}, Recall: {best_stats[1]:.3f}, F1: {best_stats[2]:.3f}')

# Confusion matrix at best threshold
best_preds = (probs_imp >= best_thresh).astype(int)
cm_imp = confusion_matrix(y_test, best_preds)
ConfusionMatrixDisplay(cm_imp, display_labels=['Non-pop', 'Pop']).plot()
plt.title(f'Improved Keras FFNN Confusion Matrix (thr={best_thresh:.2f})')
plt.show()


## 4. Final Models & Hyperparameter Tuning


### Deeper Architecture


## 5. Enhanced Models with Advanced Techniques


In [ ]:
# Enhanced Model: Better Architecture + Feature Engineering + Hyperparameter Tuning
# FIXED: Uses EXACT SAME train/test split indices as other models

import scipy.sparse as sp

# Additional feature engineering on the SAME dataframe
df_enhanced = df.copy()

# Polynomial features for key interactions
df_enhanced['popularity_squared'] = df_enhanced['spotify_popularity'] ** 2
df_enhanced['tempo_squared'] = df_enhanced['tempo_bpm_synth'] ** 2
df_enhanced['popularity_x_tempo'] = df_enhanced['spotify_popularity'] * df_enhanced['tempo_bpm_synth']

# Binning features (handle NaN from cut)
df_enhanced['popularity_bin'] = pd.cut(df_enhanced['spotify_popularity'], bins=5, labels=False).fillna(-1).astype(int)
df_enhanced['tempo_bin'] = pd.cut(df_enhanced['tempo_bpm_synth'], bins=5, labels=False).fillna(-1).astype(int)

# Prepare enhanced features (add new polynomial/binned features)
enhanced_numeric = numeric_features + ['is_explicit_binary', 'release_month', 'release_decade',
                                      'popularity_x_year', 'tempo_x_year',
                                      'popularity_squared', 'tempo_squared', 'popularity_x_tempo',
                                      'popularity_bin', 'tempo_bin'] + derived_features

X_num_enhanced = df_enhanced[enhanced_numeric].copy()
X_cat_enhanced = df_enhanced[[cat_feature]].copy()

# Scale enhanced features on FULL dataset (same as original)
scaler_enhanced = StandardScaler()
X_num_enhanced_scaled = scaler_enhanced.fit_transform(X_num_enhanced)

# Build enhanced feature matrix
X_tab_enhanced = pd.concat([
    pd.DataFrame(X_num_enhanced_scaled, columns=X_num_enhanced.columns),
    X_cat_dummies.reset_index(drop=True),
], axis=1)

X_enhanced_full = sp.csr_matrix(X_tab_enhanced.values.astype('float32'))
print(f'Enhanced feature dimension: {X_enhanced_full.shape[1]}')

# CRITICAL FIX: Use EXACT SAME split as original models
# Since we use same random_state=42 and stratify=y, splits will be identical
# But to be 100% sure, we recreate the split with same parameters
X_train_enh_full, X_temp_enh, y_train_enh, y_temp_enh = train_test_split(
    X_enhanced_full, y, test_size=0.3, random_state=42, stratify=y
)
X_val_enh, X_test_enh, y_val_enh, y_test_enh = train_test_split(
    X_temp_enh, y_temp_enh, test_size=0.5, random_state=42, stratify=y_temp_enh
)

# Convert to numpy
X_train_enh_np = X_train_enh_full.toarray().astype('float32')
X_val_enh_np = X_val_enh.toarray().astype('float32')
X_test_enh_np = X_test_enh.toarray().astype('float32')

# VERIFY: Test sets should be identical (same random_state, same stratify)
assert np.array_equal(y_test, y_test_enh), "Test sets don't match! This is a bug."
assert np.array_equal(y_train, y_train_enh), "Train sets don't match! This is a bug."
print(f'✅ Verified: Using SAME train/test split as other models')
print(f'  Train: {X_train_enh_np.shape[0]} samples')
print(f'  Val: {X_val_enh_np.shape[0]} samples')
print(f'  Test: {X_test_enh_np.shape[0]} samples')

input_dim_enhanced = X_train_enh_np.shape[1]
print(f'Input dimension (enhanced): {input_dim_enhanced}')


In [ ]:
# Enhanced FFNN with Better Architecture + Batch Normalization
# Improved: Larger network to match deeper model performance

reg_enhanced = tfk.regularizers.l2(1e-4)

model_enhanced = tfk.Sequential([
    tfl.Input(shape=(input_dim_enhanced,)),
    tfl.Dense(128, activation='relu', kernel_regularizer=reg_enhanced),
    tfl.BatchNormalization(),
    tfl.Dropout(0.4),
    tfl.Dense(64, activation='relu', kernel_regularizer=reg_enhanced),
    tfl.BatchNormalization(),
    tfl.Dropout(0.3),
    tfl.Dense(32, activation='relu', kernel_regularizer=reg_enhanced),
    tfl.BatchNormalization(),
    tfl.Dropout(0.2),
    tfl.Dense(1, activation='sigmoid')
])

model_enhanced.compile(
    optimizer=tfk.optimizers.Adam(learning_rate=1e-3),  # Same as deeper model
    loss=tfk.losses.BinaryCrossentropy(),
    metrics=[tfk.metrics.BinaryAccuracy(name='accuracy')]
)

model_enhanced.summary()


In [ ]:
# Train enhanced model with better callbacks + class weights

# Compute class weights for imbalanced labels (like deeper model)
from sklearn.utils import class_weight
classes_enh = np.unique(y_train_enh)
weights_enh = class_weight.compute_class_weight(class_weight='balanced', classes=classes_enh, y=y_train_enh)
class_weight_dict_enh = {int(c): float(w) for c, w in zip(classes_enh, weights_enh)}
print(f'Class weights: {class_weight_dict_enh}')

callbacks_enhanced = [
    tfkc.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    tfkc.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
    tfkc.ModelCheckpoint('best_enhanced_model.h5', monitor='val_loss', save_best_only=True, verbose=0)
]

history_enhanced = model_enhanced.fit(
    X_train_enh_np, y_train_enh,
    validation_data=(X_val_enh_np, y_val_enh),
    epochs=200,  # More epochs for better convergence
    batch_size=256,  # Larger batch for stability
    verbose=1,
    callbacks=callbacks_enhanced,
    class_weight=class_weight_dict_enh  # Handle class imbalance
)


In [ ]:
# Evaluate enhanced model

enhanced_probs = model_enhanced.predict(X_test_enh_np).ravel()
enhanced_preds = (enhanced_probs >= 0.5).astype(int)

print('\nEnhanced Model Classification Report:')
print(classification_report(y_test, enhanced_preds, digits=3))

enhanced_auc = roc_auc_score(y_test, enhanced_probs)
print(f'Enhanced Model ROC AUC: {enhanced_auc:.3f}')

cm_enhanced = confusion_matrix(y_test, enhanced_preds)
ConfusionMatrixDisplay(cm_enhanced, display_labels=['Non-pop', 'Pop']).plot()
plt.title('Enhanced Model Confusion Matrix')
plt.show()


### Model Improvements Summary

**Enhanced Model includes:**

1. **Better Feature Engineering:**
   - Polynomial features (popularity², tempo², popularity×tempo)
   - Binned features (popularity_bin, tempo_bin)
   - More interaction features

2. **Improved Architecture:**
   - Larger network: 64→32→16 neurons
   - Batch Normalization for stable training
   - Progressive dropout (0.4→0.3→0.2)

3. **Better Training:**
   - Lower learning rate (5e-4)
   - Model checkpointing
   - More epochs (150) with early stopping
   - Smaller batch size (128) for better gradients

**Expected improvements:**
- Better feature representation
- More stable training
- Higher AUC (target: >0.90)


In [ ]:
# Deeper FFNN with enhanced architecture
input_dim = X_train_np.shape[1]
print('Input dimension:', input_dim)

deeper_model = tfk.Sequential([
    tfl.Input(shape=(input_dim,)),
    tfl.Dense(128, activation='relu', kernel_regularizer=tfk.regularizers.l2(1e-4)),
    tfl.BatchNormalization(),
    tfl.Dropout(0.4),
    tfl.Dense(64, activation='relu', kernel_regularizer=tfk.regularizers.l2(1e-4)),
    tfl.BatchNormalization(),
    tfl.Dropout(0.3),
    tfl.Dense(32, activation='relu', kernel_regularizer=tfk.regularizers.l2(1e-4)),
    tfl.BatchNormalization(),
    tfl.Dropout(0.2),
    tfl.Dense(16, activation='relu'),
    tfl.Dense(1, activation='sigmoid'),
])

deeper_model.compile(
    optimizer=tfk.optimizers.Adam(learning_rate=1e-3),
    loss=tfk.losses.BinaryCrossentropy(),
    metrics=[tfk.metrics.BinaryAccuracy(name='accuracy')]
)

deeper_model.summary()

# Compute class weights for imbalanced labels
from sklearn.utils import class_weight
classes = np.unique(y_train)
weights = class_weight.compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = {int(c): float(w) for c, w in zip(classes, weights)}

callbacks_deeper = [
    tfkc.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    tfkc.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1),
]

history_deeper = deeper_model.fit(
    X_train_np, y_train,
    validation_data=(X_val_np, y_val),
    epochs=100,
    batch_size=256,
    verbose=1,
    callbacks=callbacks_deeper,
    class_weight=class_weight_dict,
)

# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_deeper.history['loss'], label='Train loss')
plt.plot(history_deeper.history['val_loss'], label='Val loss')
plt.xlabel('Epoch')
plt.ylabel('Binary cross-entropy')
plt.title('Deeper FFNN Training vs Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
if 'accuracy' in history_deeper.history:
    plt.plot(history_deeper.history['accuracy'], label='Train accuracy')
    plt.plot(history_deeper.history.get('val_accuracy', []), label='Val accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Deeper FFNN Training vs Validation Accuracy')
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Evaluate deeper model on test set
deeper_probs = deeper_model.predict(X_test_np).ravel()
deeper_preds = (deeper_probs >= 0.5).astype(int)

print('\nDeeper FFNN classification report (0 = non-pop, 1 = pop):')
print(classification_report(y_test, deeper_preds, digits=3))

try:
    deeper_auc = roc_auc_score(y_test, deeper_probs)
    print(f'Deeper FFNN ROC AUC: {deeper_auc:.3f}')
except Exception as e:
    print('Could not compute ROC AUC:', e)

cm_deeper = confusion_matrix(y_test, deeper_preds)
ConfusionMatrixDisplay(cm_deeper, display_labels=['Non-pop', 'Pop']).plot()
plt.title('Deeper FFNN Confusion Matrix')
plt.show()


In [ ]:
# Show all feature names being used
print('=== ALL FEATURES IN THE MODEL ===\n')
print(f'Numeric features ({len(X_num.columns)}):')
for feat in X_num.columns:
    print(f'  - {feat}')

print(f'\nCategorical features - one-hot encoded ({X_cat_dummies.shape[1]}):')
for feat in X_cat_dummies.columns:
    print(f'  - {feat}')

print(f'\nTF-IDF Genre features: DISABLED (removed to prevent data leakage)')

print(f'\nTotal features: {X_full.shape[1]}')


In [ ]:
# Compare FFNN Models: ROC Curves
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(8, 6))

# Baseline FFNN
try:
    if 'probs' in globals():
        fpr_base, tpr_base, _ = roc_curve(y_test, probs)
        roc_base = auc(fpr_base, tpr_base)
        plt.plot(fpr_base, tpr_base, label=f'Baseline FFNN (AUC={roc_base:.3f})')
except Exception as e:
    print('Could not plot baseline FFNN ROC:', e)

# Improved FFNN
try:
    if 'probs_imp' in globals():
        fpr_imp, tpr_imp, _ = roc_curve(y_test, probs_imp)
        roc_imp = auc(fpr_imp, tpr_imp)
        plt.plot(fpr_imp, tpr_imp, label=f'Improved FFNN (AUC={roc_imp:.3f})')
except Exception as e:
    print('Could not plot improved FFNN ROC:', e)

# Deeper FFNN
try:
    if 'deeper_probs' in globals():
        fpr_deep, tpr_deep, _ = roc_curve(y_test, deeper_probs)
        roc_deep = auc(fpr_deep, tpr_deep)
        plt.plot(fpr_deep, tpr_deep, label=f'Deeper FFNN (AUC={roc_deep:.3f})')
except Exception as e:
    print('Could not plot deeper FFNN ROC:', e)

# Enhanced FFNN
try:
    if 'enhanced_probs' in globals():
        fpr_enh, tpr_enh, _ = roc_curve(y_test, enhanced_probs)
        roc_enh = auc(fpr_enh, tpr_enh)
        plt.plot(fpr_enh, tpr_enh, label=f'Enhanced FFNN (AUC={roc_enh:.3f})', linewidth=2)
except Exception as e:
    print('Could not plot enhanced FFNN ROC:', e)

# Random baseline
plt.plot([0, 1], [0, 1], 'k--', label='Random', alpha=0.5)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves: FFNN Models Comparison')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()
